In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import sqlalchemy
from sqlalchemy import create_engine # database connection
import sqlite3
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.display import display, clear_output
from sklearn.naive_bayes import MultinomialNB
from datetime import datetime
from sklearn.metrics import accuracy_score

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
# print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [2]:
# engine  = create_engine("sqlite:///../input/database.sqlite")
with sqlite3.connect('database.sqlite') as engine:
#    matches = pd.read_sql_query('SELECT * FROM Match where league_id = 1729 and season in ("2010/2011", "2011/2012", "2012/2013", "2013/2014", "2014/2015", "2015/2016");'
#                                          , engine)
    matches = pd.read_sql_query('SELECT * FROM Match where league_id = 1729 ;', engine)


matches = matches[matches.columns[:11]]
teams = pd.read_sql_query('SELECT * FROM Team;', engine)

# Add team names & tidy up
matches = pd.merge(left=matches, right=teams, how='left', left_on='home_team_api_id', right_on='team_api_id')
matches = matches.drop(['country_id','league_id', 'home_team_api_id', 'id_y', 'team_api_id', 'team_short_name'], axis=1)
matches.rename(columns={'id_x':'match_id', 'team_long_name':'home_team'}, inplace=True)
matches = pd.merge(left=matches, right=teams, how='left', left_on='away_team_api_id', right_on='team_api_id')
matches = matches.drop(['id', 'match_api_id', 'away_team_api_id','team_api_id', 'team_short_name'], axis=1)
matches.rename(columns={'id_x':'match_id', 'team_long_name':'away_team'}, inplace=True)

matches.tail(20)
# matches.info()

,match_id,season,stage,date,home_team_goal,away_team_goal,team_fifa_api_id_x,home_team,team_fifa_api_id_y,away_team
3020,4749,2015/2016,8,2015-10-04 00:00:00,3,0,1.0,Arsenal,11.0,Manchester United
3021,4750,2015/2016,8,2015-10-03 00:00:00,0,1,2.0,Aston Villa,1806.0,Stoke City
3022,4751,2015/2016,8,2015-10-03 00:00:00,1,1,1943.0,Bournemouth,1795.0,Watford
3023,4752,2015/2016,8,2015-10-03 00:00:00,1,3,5.0,Chelsea,17.0,Southampton
3024,4753,2015/2016,8,2015-10-03 00:00:00,2,0,1799.0,Crystal Palace,109.0,West Bromwich Albion
3025,4754,2015/2016,8,2015-10-04 00:00:00,1,1,7.0,Everton,9.0,Liverpool
3026,4755,2015/2016,8,2015-10-03 00:00:00,6,1,10.0,Manchester City,13.0,Newcastle United
3027,4756,2015/2016,8,2015-10-03 00:00:00,1,2,1792.0,Norwich City,95.0,Leicester City
3028,4757,2015/2016,8,2015-10-03 00:00:00,2,2,106.0,Sunderland,19.0,West Ham United
3029,4758,2015/2016,8,2015-10-04 00:00:00,2,2,1960.0,Swansea City,18.0,Tottenham Hotspur


In [3]:
def convert_date_to_code16(matches_data_16):
    date = []
    for i in matches_data_16['date']:
        date.append(i)
    x = date[0]
    d = datetime.strptime(x, "%d/%m/%y").date()
    dict2 = {}
    base = 42595
    for i in date:
        date_l = datetime.strptime(i, "%d/%m/%y").date()
        date_dif = date_l - d
        dict2[date_l] = str(base + date_dif.days)
    t = []
    for i in range(len(date)):
        t.append(dict2[datetime.strptime(date[i], '%d/%m/%y').date()])
    matches_data_16['date'] = list(map((lambda i: t[i]), range(len(t))))
    return matches_data_16

In [4]:
def convert_date_to_code17(matches_data_17):
    stage = []
    for i in matches_data_17['stage']:
        stage.append(i)
    date = []
    for i in matches_data_17['date']:
        date.append(i)
    dict_date = {}
    for i, j in zip(stage, date):
        dict_date[i] = datetime.strptime(j, "%d/%m/%y").date()
    dict2 = {}
    base = 42959
    for i in stage:
        if i == 1:
            dict2[i] = base
        else:
            date_dif = dict_date[i] - dict_date[i - 1]
            if date_dif.days < 13:
                dict2[i] = dict2[i - 1] + 7
            if date_dif.days >= 13:
                dict2[i] = dict2[i - 1] + 14
    t = []
    for i in stage:
        t.append(str(dict2[i]))
    matches_data_17['date'] = list(map((lambda i: t[i]), range(len(t))))
    return matches_data_17

In [5]:
def convert_team_name(last_season):
    for index, row in last_season.iterrows():
        if row['home_team'] == 'Man United':
            last_season.loc[index, 'home_team'] = 'Manchester United'
        if row['home_team'] == 'Man City':
            last_season.loc[index, 'home_team'] = 'Manchester City'
        if row['home_team'] == 'Leicester':
            last_season.loc[index, 'home_team'] = 'Leicester City'
        if row['home_team'] == 'West Brom':
            last_season.loc[index, 'home_team'] = 'West Bromwich Albion'
        if row['home_team'] == 'Newcastle':
            last_season.loc[index, 'home_team'] = 'Newcastle United'
        if row['home_team'] == 'Stoke':
            last_season.loc[index, 'home_team'] = 'Stoke City'
        if row['home_team'] == 'Swansea':
            last_season.loc[index, 'home_team'] = 'Swansea City'
        if row['home_team'] == 'West Ham':
            last_season.loc[index, 'home_team'] = 'West Ham United'
        if row['home_team'] == 'Tottenham':
            last_season.loc[index, 'home_team'] = 'Tottenham Hotspur'
        if row['home_team'] == 'Hull':
            last_season.loc[index, 'home_team'] = 'Hull City'
        if row['home_team'] == 'Brighton & Hove Albion':
            last_season.loc[index, 'home_team'] = 'Brighton'
        if row['home_team'] == 'Huddersfield Town':
            last_season.loc[index, 'home_team'] = 'Huddersfield'

        if row['away_team'] == 'Man United':
            last_season.loc[index, 'away_team'] = 'Manchester United'
        if row['away_team'] == 'Man City':
            last_season.loc[index, 'away_team'] = 'Manchester City'
        if row['away_team'] == 'Leicester':
            last_season.loc[index, 'away_team'] = 'Leicester City'
        if row['away_team'] == 'West Brom':
            last_season.loc[index, 'away_team'] = 'West Bromwich Albion'
        if row['away_team'] == 'Newcastle':
            last_season.loc[index, 'away_team'] = 'Newcastle United'
        if row['away_team'] == 'Stoke':
            last_season.loc[index, 'away_team'] = 'Stoke City'
        if row['away_team'] == 'Swansea':
            last_season.loc[index, 'away_team'] = 'Swansea City'
        if row['away_team'] == 'West Ham':
            last_season.loc[index, 'away_team'] = 'West Ham United'
        if row['away_team'] == 'Tottenham':
            last_season.loc[index, 'away_team'] = 'Tottenham Hotspur'
        if row['away_team'] == 'Hull':
            last_season.loc[index, 'away_team'] = 'Hull City'
        if row['away_team'] == 'Brighton & Hove Albion':
            last_season.loc[index, 'away_team'] = 'Brighton'
        if row['away_team'] == 'Huddersfield Town':
            last_season.loc[index, 'away_team'] = 'Huddersfield'
    return last_season

In [6]:
def create_match_id():
    match_id_num = []
    match_id_num2 = []
    for i in range(1, 11):
        match_id_num.append(i)
    for i in match_id_num:
        match_id_num2.append(i + int(last_season[-1:]['match_id']))
    return match_id_num2

In [7]:
with sqlite3.connect('database2.sqlite') as con:
    matches_data_16 = pd.read_sql('select * from match;', con)
    matches_data_17 = pd.read_sql('select * from match17;', con)
    matches_schedule = pd.read_sql('select * from match_schedule;', con)
    team_rating = pd.read_sql('select * from team_rating;', con)
# matches_data_16['date'] = pd.to_datetime(matches_data_16['date'])

matches_data_16 = convert_date_to_code16(matches_data_16)
matches_data_17 = convert_date_to_code17(matches_data_17)
matches_schedule = convert_date_to_code17(matches_schedule)
last_season = pd.concat([matches_data_16, matches_data_17])
# int(last_season[-1:]['match_id'])
# matches_schedule.set_index('index', inplace=True)
# matches_schedule.columns = ['date', 'home_team_goal', 'away_team_goal', 'home_team', 'away_team', 'season', 'stage']
# matches_schedule = convert_team_name(matches_schedule)
# matches_schedule.head(190)
match_id = create_match_id()
# match_id
last_season.tail()

,match_id,season,stage,date,home_team_goal,away_team_goal,home_team,away_team
245,7245,2017/2018,25,43148,3,0,Man City,West Brom
246,7246,2017/2018,25,43148,1,1,Newcastle,Burnley
247,7247,2017/2018,25,43148,1,1,Southampton,Brighton
248,7248,2017/2018,25,43148,0,0,Stoke,Watford
249,7249,2017/2018,25,43148,2,0,Tottenham,Man United


In [8]:
# Parameters to change depending on season and wek we are running for
this_season = '2017/2018'
this_week = int(matches_data_17[-1:]['stage']) + 1
train_ratio = .8
model_weight = .8
this_week

26

In [9]:
matches_schedule = matches_schedule.loc[matches_schedule['stage'] == this_week]
matches_schedule['match_id'] = match_id
matches_schedule

,match_id,date,home_team,away_team,season,stage
250,7250,43155,Bournemouth,Stoke City,2017/2018,26
251,7251,43155,Arsenal,Everton,2017/2018,26
252,7252,43155,Brighton & Hove Albion,West Ham United,2017/2018,26
253,7253,43155,Burnley,Manchester City,2017/2018,26
254,7254,43155,Crystal Palace,Newcastle United,2017/2018,26
255,7255,43155,Leicester City,Swansea,2017/2018,26
256,7256,43155,Liverpool,Tottenham Hotspur,2017/2018,26
257,7257,43155,Manchester United,Huddersfield Town,2017/2018,26
258,7258,43155,Watford,Chelsea,2017/2018,26
259,7259,43155,West Bromwich Albion,Southampton,2017/2018,26


In [10]:
# Add to full training data to predict current season
matches = pd.concat([matches, last_season, matches_schedule])
matches = matches.reset_index(drop=True)
convert_team_name(matches)
# matches

# Create a full set of match data that can be used with feature engineering later
full_matches = matches.copy()
# full_matches.match_id.fillna(last_season['match_id'][-1:] + 1)
full_matches

,away_team,away_team_goal,date,home_team,home_team_goal,match_id,season,stage,team_fifa_api_id_x,team_fifa_api_id_y
0,Newcastle United,1.0,2008-08-17 00:00:00,Manchester United,1.0,1729,2008/2009,1,11.0,13.0
1,West Bromwich Albion,0.0,2008-08-16 00:00:00,Arsenal,1.0,1730,2008/2009,1,1.0,109.0
2,Liverpool,1.0,2008-08-16 00:00:00,Sunderland,0.0,1731,2008/2009,1,106.0,9.0
3,Wigan Athletic,1.0,2008-08-16 00:00:00,West Ham United,2.0,1732,2008/2009,1,19.0,1917.0
4,Manchester City,2.0,2008-08-17 00:00:00,Aston Villa,4.0,1733,2008/2009,1,2.0,10.0
5,Blackburn Rovers,3.0,2008-08-16 00:00:00,Everton,2.0,1734,2008/2009,1,7.0,3.0
6,Tottenham Hotspur,1.0,2008-08-16 00:00:00,Middlesbrough,2.0,1735,2008/2009,1,12.0,18.0
7,Stoke City,1.0,2008-08-16 00:00:00,Bolton Wanderers,3.0,1736,2008/2009,1,4.0,1806.0
8,Fulham,1.0,2008-08-16 00:00:00,Hull City,2.0,1737,2008/2009,1,1952.0,144.0
9,Portsmouth,0.0,2008-08-17 00:00:00,Chelsea,4.0,1738,2008/2009,1,5.0,1790.0


In [11]:
full_matches.drop(matches[matches.season == '2008/2009'].index, inplace=True)
full_matches.drop(matches[matches.season == '2009/2010'].index, inplace=True)
full_matches.drop(matches[matches.season == '2010/2011'].index, inplace=True)
full_matches.drop(matches[matches.season == '2011/2012'].index, inplace=True)

full_matches

,away_team,away_team_goal,date,home_team,home_team_goal,match_id,season,stage,team_fifa_api_id_x,team_fifa_api_id_y
1520,Sunderland,0.0,2012-08-18 00:00:00,Arsenal,0.0,3249,2012/2013,1,1.0,106.0
1521,Manchester United,0.0,2012-08-20 00:00:00,Everton,1.0,3250,2012/2013,1,7.0,11.0
1522,Norwich City,0.0,2012-08-18 00:00:00,Fulham,5.0,3251,2012/2013,1,144.0,1792.0
1523,Southampton,2.0,2012-08-19 00:00:00,Manchester City,3.0,3252,2012/2013,1,10.0,17.0
1524,Tottenham Hotspur,1.0,2012-08-18 00:00:00,Newcastle United,2.0,3253,2012/2013,1,13.0,18.0
1525,Swansea City,5.0,2012-08-18 00:00:00,Queens Park Rangers,0.0,3254,2012/2013,1,15.0,1960.0
1526,Stoke City,1.0,2012-08-18 00:00:00,Reading,1.0,3255,2012/2013,1,1793.0,1806.0
1527,Liverpool,0.0,2012-08-18 00:00:00,West Bromwich Albion,3.0,3256,2012/2013,1,109.0,9.0
1528,Aston Villa,0.0,2012-08-18 00:00:00,West Ham United,1.0,3257,2012/2013,1,19.0,2.0
1529,Chelsea,2.0,2012-08-19 00:00:00,Wigan Athletic,0.0,3258,2012/2013,1,1917.0,5.0


full_matches : ข้อมูลการแข่งขันทั้งหมดรวมอาทิตย์ต่อไปด้วย

In [12]:
# State the result as home or away win/draw/lose - 6 possibilites
# Create a binary result
def determine_result(match_list):
    match_list['home_win'] = np.where(match_list['home_team_goal'] > match_list['away_team_goal'], 1, 0)
    match_list['home_draw'] = np.where(match_list['home_team_goal'] == match_list['away_team_goal'], 1, 0)
    match_list['home_lose'] = np.where(match_list['home_team_goal'] < match_list['away_team_goal'], 1, 0)
    match_list['away_win'] = np.where(match_list['home_team_goal'] < match_list['away_team_goal'], 1, 0)
    match_list['away_draw'] = np.where(match_list['home_team_goal'] == match_list['away_team_goal'], 1, 0)
    match_list['away_lose'] = np.where(match_list['home_team_goal'] > match_list['away_team_goal'], 1, 0)

# Set up the matches data how I need it

# Add binary feature for W/D/L home and away
determine_result(full_matches)

# Sort in date order
full_matches.sort_values(by='date', inplace=True)

full_matches.head()

,away_team,away_team_goal,date,home_team,home_team_goal,match_id,season,stage,team_fifa_api_id_x,team_fifa_api_id_y,home_win,home_draw,home_lose,away_win,away_draw,away_lose
1520,Sunderland,0.0,2012-08-18 00:00:00,Arsenal,0.0,3249,2012/2013,1,1.0,106.0,0,1,0,0,1,0
1527,Liverpool,0.0,2012-08-18 00:00:00,West Bromwich Albion,3.0,3256,2012/2013,1,109.0,9.0,1,0,0,0,0,1
1526,Stoke City,1.0,2012-08-18 00:00:00,Reading,1.0,3255,2012/2013,1,1793.0,1806.0,0,1,0,0,1,0
1525,Swansea City,5.0,2012-08-18 00:00:00,Queens Park Rangers,0.0,3254,2012/2013,1,15.0,1960.0,0,0,1,1,0,0
1528,Aston Villa,0.0,2012-08-18 00:00:00,West Ham United,1.0,3257,2012/2013,1,19.0,2.0,1,0,0,0,0,1


new_team : ข้อมูลทีมที่เลื่อนชั้น พร้อมฤดูกาล

In [13]:
# Cope with newly promoted teams with limited or no stats
team_data = {'team':['West Bromwich Albion', 'Stoke City', 'Hull City',
                     'Wolverhampton Wanderers', 'Birmingham City', 'Burnley',
                     'Newcastle United', 'West Bromwich Albion', 'Blackpool',
                     'Queens Park Rangers', 'Norwich City', 'Swansea City',
                     'Reading', 'Southampton', 'West Ham United',
                     'Cardiff City', 'Crystal Palace', 'Hull City',
                     'Leicester City', 'Burnley', 'Queens Park Rangers',
                     'Bournemouth', 'Watford', 'Norwich City',
                     'Burnley', 'Middlesbrough', 'Hull City',
                     'Brighton', 'Newcastle United', 'Huddersfield'
                    ],
             'season':["2008/2009", "2008/2009", "2008/2009",
                       "2009/2010", "2009/2010", "2009/2010",
                       "2010/2011", "2010/2011", "2010/2011",
                       "2011/2012", "2011/2012", "2011/2012",
                       "2012/2013", "2012/2013", "2012/2013",
                       "2013/2014", "2013/2014", "2013/2014",
                       "2014/2015", "2014/2015", "2014/2015",
                       "2015/2016", "2015/2016", "2015/2016",
                       "2016/2017", "2016/2017", "2016/2017",
                       "2017/2018", "2017/2018", "2017/2018"
                      ]
            }
new_teams = pd.DataFrame(team_data, columns=['team', 'season'])
new_teams.head()

,team,season
0,West Bromwich Albion,2008/2009
1,Stoke City,2008/2009
2,Hull City,2008/2009
3,Wolverhampton Wanderers,2009/2010
4,Birmingham City,2009/2010


full_match_features : dummy ข้อมูลของสถิติเก่าๆ

In [14]:
full_match_features = pd.DataFrame(full_matches[['season', 'stage']])
                                   #,
                                   #columns=[['season', 'stage']])
full_match_features.head()

,season,stage
1520,2012/2013,1
1527,2012/2013,1
1526,2012/2013,1
1525,2012/2013,1
1528,2012/2013,1


In [15]:
team_rating.drop('team_rating_id', axis=1, inplace=True)
team_rating2 = team_rating.copy()
team_rating = team_rating.rename(columns = {'team_name': 'home_team'})
team_rating2 = team_rating2.rename(columns = {'team_name': 'away_team'})

In [16]:
full_matches = pd.merge(full_matches, team_rating, how='left', on=['season', 'home_team'])
full_matches = full_matches.rename(columns = {'team_rating': 'home_team_rating'})
full_matches = pd.merge(full_matches, team_rating2, how='left', on=['season', 'away_team'])
full_matches = full_matches.rename(columns = {'team_rating': 'away_team_rating'})
full_matches

,away_team,away_team_goal,date,home_team,home_team_goal,match_id,season,stage,team_fifa_api_id_x,team_fifa_api_id_y,home_win,home_draw,home_lose,away_win,away_draw,away_lose,home_team_rating,away_team_rating
0,Sunderland,0.0,2012-08-18 00:00:00,Arsenal,0.0,3249,2012/2013,1,1.0,106.0,0,1,0,0,1,0,80,75
1,Liverpool,0.0,2012-08-18 00:00:00,West Bromwich Albion,3.0,3256,2012/2013,1,109.0,9.0,1,0,0,0,0,1,74,79
2,Stoke City,1.0,2012-08-18 00:00:00,Reading,1.0,3255,2012/2013,1,1793.0,1806.0,0,1,0,0,1,0,71,75
3,Swansea City,5.0,2012-08-18 00:00:00,Queens Park Rangers,0.0,3254,2012/2013,1,15.0,1960.0,0,0,1,1,0,0,75,74
4,Aston Villa,0.0,2012-08-18 00:00:00,West Ham United,1.0,3257,2012/2013,1,19.0,2.0,1,0,0,0,0,1,74,75
5,Norwich City,0.0,2012-08-18 00:00:00,Fulham,5.0,3251,2012/2013,1,144.0,1792.0,1,0,0,0,0,1,77,73
6,Tottenham Hotspur,1.0,2012-08-18 00:00:00,Newcastle United,2.0,3253,2012/2013,1,13.0,18.0,1,0,0,0,0,1,77,80
7,Southampton,2.0,2012-08-19 00:00:00,Manchester City,3.0,3252,2012/2013,1,10.0,17.0,1,0,0,0,0,1,82,73
8,Chelsea,2.0,2012-08-19 00:00:00,Wigan Athletic,0.0,3258,2012/2013,1,1917.0,5.0,0,0,1,1,0,0,74,81
9,Manchester United,0.0,2012-08-20 00:00:00,Everton,1.0,3250,2012/2013,1,7.0,11.0,1,0,0,0,0,1,76,82


In [17]:
# Convert home & team into a binary feature, ie Arsenal_h or Arsenal_a
# Need all seasons data for team binary feature
full_match_features = pd.DataFrame(full_matches[['season', 'stage']]) # เอา column ไหนมาบ้าง
#                                    columns=[['season', 'stage']])

full_match_features = pd.concat([full_match_features, pd.get_dummies(full_matches['home_team']).rename(columns=lambda x: str(x) + '_h')],
                                axis=1)
full_match_features = pd.concat([full_match_features, pd.get_dummies(full_matches['away_team']).rename(columns=lambda x: str(x) + '_a')],
                         axis=1)

full_match_features.head() # ข้อมูลสถิติเก่าๆ

,season,stage,Arsenal_h,Aston Villa_h,Bournemouth_h,Brighton_h,Burnley_h,Cardiff City_h,Chelsea_h,Crystal Palace_h,...,Reading_a,Southampton_a,Stoke City_a,Sunderland_a,Swansea City_a,Tottenham Hotspur_a,Watford_a,West Bromwich Albion_a,West Ham United_a,Wigan Athletic_a
0,2012/2013,1,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,2012/2013,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2012/2013,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,2012/2013,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,2012/2013,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


full_match_features_2 : ข้อมูล rating team เก่าๆ

In [18]:
# DataFrame team rating
full_match_features_2 = pd.DataFrame(full_matches[['season', 'stage', 'home_team_rating', 'away_team_rating']])

full_match_features_2.head() # ข้อมูล team rating

,season,stage,home_team_rating,away_team_rating
0,2012/2013,1,80,75
1,2012/2013,1,74,79
2,2012/2013,1,71,75
3,2012/2013,1,75,74
4,2012/2013,1,74,75


In [19]:
# Function to determine whether the highest prediction is for win/draw/lose

def predict_home_result(match):
    if (match['win'] >= match['draw']) & (match['win'] >= match['lose']):
        return 'win' # Favour a home win if probability equal
    elif (match['lose'] > match['win']) & (match['lose'] > match['draw']):
        return 'lose'
    else:
        return 'draw'

train_match_features : ข้อมูล dummy สถิติเก่าๆ ไว้ train model 

In [20]:
# To predict this season (1, this week only, remove this week from training set
train_match_features = full_match_features.loc[:int(full_match_features.shape[0] * train_ratio )].copy()

train_match_features.drop(['season'], axis=1, inplace=True)
train_match_features.tail()

,stage,Arsenal_h,Aston Villa_h,Bournemouth_h,Brighton_h,Burnley_h,Cardiff City_h,Chelsea_h,Crystal Palace_h,Everton_h,...,Reading_a,Southampton_a,Stoke City_a,Sunderland_a,Swansea City_a,Tottenham Hotspur_a,Watford_a,West Bromwich Albion_a,West Ham United_a,Wigan Athletic_a
1724,21,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1725,21,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1726,21,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1727,21,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1728,21,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


train_match_features_2 : ข้อมูล rating team เก่าๆ ไว้ train model

In [21]:
# To predict this season (1, this week only, remove this week from training set
train_match_features_2 = full_match_features_2.loc[:int(full_match_features.shape[0] * train_ratio  )].copy()

train_match_features_2.drop(['season', 'stage'], axis=1, inplace=True)
train_match_features_2.tail()

,home_team_rating,away_team_rating
1724,75,74
1725,76,78
1726,74,78
1727,77,83
1728,80,83


In [22]:
# Get the target results for training

# Function to determine whether result is a win/draw/lose
def determine_home_result(match):
    if match['home_team_goal'] > match['away_team_goal']:
        return 'win'
    elif match['home_team_goal'] < match['away_team_goal']:
        return 'lose'
    else:
        return 'draw'

train_matches : ข้อมูล train ทั้งหมดพร้อม label แล้ว

In [23]:
#  Add the home team result column to the matches dataframe
full_matches['home_team_result'] = full_matches.apply(determine_home_result, axis=1)

# To predict this season, this week, remove latest week from training results
train_matches = full_matches.loc[:int(full_matches.shape[0] *  train_ratio  )].copy()

targets = train_matches['home_team_result'].values
train_matches.tail()

,away_team,away_team_goal,date,home_team,home_team_goal,match_id,season,stage,team_fifa_api_id_x,team_fifa_api_id_y,home_win,home_draw,home_lose,away_win,away_draw,away_lose,home_team_rating,away_team_rating,home_team_result
1724,Bournemouth,1.0,42749,Hull City,3.0,6201,2016/2017,21,NaN,NaN,1,0,0,0,0,1,75,74,win
1725,Stoke City,3.0,42749,Sunderland,1.0,6203,2016/2017,21,NaN,NaN,0,0,1,1,0,0,76,78,lose
1726,Southampton,0.0,42749,Burnley,1.0,6200,2016/2017,21,NaN,NaN,1,0,0,0,0,1,74,78,win
1727,Arsenal,4.0,42749,Swansea City,0.0,6204,2016/2017,21,NaN,NaN,0,0,1,1,0,0,77,83,lose
1728,Manchester City,0.0,42750,Everton,4.0,6208,2016/2017,21,NaN,NaN,1,0,0,0,0,1,80,83,win


test_match_features : ข้อมูล dummy สถิติเก่าๆ ไว้ test model

In [24]:
# Get the test matches in correct format:

# Predict this season this week
test_match_features = full_match_features.loc[int(full_match_features.shape[0] * train_ratio  ):].copy()

test_match_features.drop(full_match_features.index[-10:], inplace=True)
test_match_features.drop(['season'], axis=1, inplace=True)
test_match_features

,stage,Arsenal_h,Aston Villa_h,Bournemouth_h,Brighton_h,Burnley_h,Cardiff City_h,Chelsea_h,Crystal Palace_h,Everton_h,...,Reading_a,Southampton_a,Stoke City_a,Sunderland_a,Swansea City_a,Tottenham Hotspur_a,Watford_a,West Bromwich Albion_a,West Ham United_a,Wigan Athletic_a
1728,21,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1729,21,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1730,22,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1731,22,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1732,22,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1733,22,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1734,22,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1735,22,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1736,22,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1737,22,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


test_match_features_1 : ข้อมูล dummy ของ week ถัดไป ไว้ทำนายจาก model สถิติเก่า

In [25]:
test_match_features_1 = full_match_features.iloc[-10:].copy()
test_match_features_1.drop(['season'], axis=1, inplace=True)
test_match_features_1

,stage,Arsenal_h,Aston Villa_h,Bournemouth_h,Brighton_h,Burnley_h,Cardiff City_h,Chelsea_h,Crystal Palace_h,Everton_h,...,Reading_a,Southampton_a,Stoke City_a,Sunderland_a,Swansea City_a,Tottenham Hotspur_a,Watford_a,West Bromwich Albion_a,West Ham United_a,Wigan Athletic_a
2150,26,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2151,26,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2152,26,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2153,26,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2154,26,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2155,26,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2156,26,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2157,26,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2158,26,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2159,26,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


test_match_features_2 : ข้อมูล team rating เก่าๆไว้ test model

In [26]:
test_match_features_2 = full_match_features_2.loc[int(full_match_features.shape[0] * train_ratio  ):].copy()

test_match_features_2.drop(full_match_features.index[-10:], inplace=True)
test_match_features_2.drop(['season', 'stage'], axis=1, inplace=True)
test_match_features_2

,home_team_rating,away_team_rating
1728,80,83
1729,83,81
1730,75,76
1731,74,77
1732,77,80
1733,81,77
1734,83,81
1735,75,78
1736,78,83
1737,78,78


test_match_features_2_1 : ข้อมูล next week ของ team rating ไว้ทำนายจาก model team rating

In [27]:
test_match_features_2_1 = full_match_features_2.iloc[-10:].copy()
test_match_features_2_1.drop(['season', 'stage'], axis=1, inplace=True)
test_match_features_2_1

,home_team_rating,away_team_rating
2150,77,84
2151,75,77
2152,82,80
2153,75,78
2154,76,84
2155,76,75
2156,78,75
2157,81,83
2158,84,73
2159,77,77


model_test_matches : ข้อมูลทั้งหมดไว้ test model

In [28]:
# Don't have the target results yet but I have entered dummy data of 0-0 draws. 
# Then re-run with actual scores after they are played for comparison
model_test_matches = full_matches.loc[int(full_matches.shape[0] * train_ratio  ):].copy()
# model_test_matches = full_matches.loc[(full_matches['season'] == this_season) & 
#                                      (full_matches['stage'] == this_week - 1)].copy()

model_test_matches.drop(full_match_features.index[-10:], inplace=True)
model_test_matches = model_test_matches.reset_index(drop=True)
model_test_matches

,away_team,away_team_goal,date,home_team,home_team_goal,match_id,season,stage,team_fifa_api_id_x,team_fifa_api_id_y,home_win,home_draw,home_lose,away_win,away_draw,away_lose,home_team_rating,away_team_rating,home_team_result
0,Manchester City,0.0,42750,Everton,4.0,6208,2016/2017,21,NaN,NaN,1,0,0,0,0,1,80,83,win
1,Liverpool,1.0,42750,Manchester United,1.0,6209,2016/2017,21,NaN,NaN,0,1,0,0,1,0,83,81,draw
2,Sunderland,0.0,42756,West Bromwich Albion,2.0,6216,2016/2017,22,NaN,NaN,1,0,0,0,0,1,75,76,win
3,Watford,2.0,42756,Bournemouth,2.0,6210,2016/2017,22,NaN,NaN,0,1,0,0,1,0,74,77,draw
4,Everton,1.0,42756,Crystal Palace,0.0,6211,2016/2017,22,NaN,NaN,0,0,1,1,0,0,77,80,lose
5,Swansea City,3.0,42756,Liverpool,2.0,6212,2016/2017,22,NaN,NaN,0,0,1,1,0,0,81,77,lose
6,Tottenham Hotspur,2.0,42756,Manchester City,2.0,6213,2016/2017,22,NaN,NaN,0,1,0,0,1,0,83,81,draw
7,West Ham United,3.0,42756,Middlesbrough,1.0,6214,2016/2017,22,NaN,NaN,0,0,1,1,0,0,75,78,lose
8,Manchester United,1.0,42756,Stoke City,1.0,6215,2016/2017,22,NaN,NaN,0,1,0,0,1,0,78,83,draw
9,Leicester City,0.0,42757,Southampton,3.0,6219,2016/2017,22,NaN,NaN,1,0,0,0,0,1,78,78,win


model_test_matches_1 : ข้อมูลทั้งหมดของ next week

In [29]:
model_test_matches_1 = full_matches.iloc[-10:].copy()
# model_test_matches = full_matches.loc[(full_matches['season'] == this_season) & 
#                                      (full_matches['stage'] == this_week - 1)].copy()

# model_test_matches.drop(full_match_features.index[-10:], inplace=True)
model_test_matches_1 = model_test_matches_1.reset_index(drop=True)
model_test_matches_1

,away_team,away_team_goal,date,home_team,home_team_goal,match_id,season,stage,team_fifa_api_id_x,team_fifa_api_id_y,home_win,home_draw,home_lose,away_win,away_draw,away_lose,home_team_rating,away_team_rating,home_team_result
0,Chelsea,NaN,43155,Watford,NaN,7258,2017/2018,26,NaN,NaN,0,0,0,0,0,0,77,84,draw
1,Stoke City,NaN,43155,Bournemouth,NaN,7250,2017/2018,26,NaN,NaN,0,0,0,0,0,0,75,77,draw
2,Everton,NaN,43155,Arsenal,NaN,7251,2017/2018,26,NaN,NaN,0,0,0,0,0,0,82,80,draw
3,West Ham United,NaN,43155,Brighton,NaN,7252,2017/2018,26,NaN,NaN,0,0,0,0,0,0,75,78,draw
4,Manchester City,NaN,43155,Burnley,NaN,7253,2017/2018,26,NaN,NaN,0,0,0,0,0,0,76,84,draw
5,Newcastle United,NaN,43155,Crystal Palace,NaN,7254,2017/2018,26,NaN,NaN,0,0,0,0,0,0,76,75,draw
6,Swansea City,NaN,43155,Leicester City,NaN,7255,2017/2018,26,NaN,NaN,0,0,0,0,0,0,78,75,draw
7,Tottenham Hotspur,NaN,43155,Liverpool,NaN,7256,2017/2018,26,NaN,NaN,0,0,0,0,0,0,81,83,draw
8,Huddersfield,NaN,43155,Manchester United,NaN,7257,2017/2018,26,NaN,NaN,0,0,0,0,0,0,84,73,draw
9,Southampton,NaN,43155,West Bromwich Albion,NaN,7259,2017/2018,26,NaN,NaN,0,0,0,0,0,0,77,77,draw


## Fit model

fit ของ ข้อมูลการแข่งขันเก่าๆ

In [30]:
# Train, then predict 
model = MultinomialNB()

model.fit(train_match_features.values, targets) # รับเป็น array ของสิ่งที่จะทำนายและผลเข้าไป
predicted = model.predict_proba(test_match_features.values) # รับเป็น array อาทิตที่จะทำนายต่อไป

# Format the output into a DF with columns
predicted_table = pd.DataFrame(predicted,columns=['draw', 'lose', 'win'])

# Compare predicted with test actual results 
predicted_table['predict_res'] = predicted_table.apply(predict_home_result, axis=1)
predicted_table['actual_res'] = model_test_matches['home_team_result']

# Straight comparison - count of equal / total to get %
(predicted_table[predicted_table['predict_res'] 
                 == model_test_matches['home_team_result']].count()) / model_test_matches['home_team_result'].count()

draw           0.580569
lose           0.580569
win            0.580569
predict_res    0.580569
actual_res     0.580569
dtype: float64

test model ข้อมูลการแข่งขันเก่าๆ

In [31]:
predicted_1 = model.predict_proba(test_match_features_1.values) # รับเป็น array อาทิตที่จะทำนายต่อไป

# Format the output into a DF with columns
predicted_table_1 = pd.DataFrame(predicted_1,columns=['draw', 'lose', 'win'])

# Compare predicted with test actual results 
predicted_table_1['predict_res'] = predicted_table_1.apply(predict_home_result, axis=1)
predicted_table_1['actual_res'] = model_test_matches_1['home_team_result']

# Straight comparison - count of equal / total to get %
(predicted_table_1[predicted_table_1['predict_res'] 
                 == model_test_matches_1['home_team_result']].count()) / model_test_matches_1['home_team_result'].count()

draw           0.0
lose           0.0
win            0.0
predict_res    0.0
actual_res     0.0
dtype: float64

fit ข้อมูล team rating เก่าๆ

In [32]:
# Train, then predict 
model_2 = MultinomialNB()

model_2.fit(train_match_features_2.values, targets) # รับเป็น array ของสิ่งที่จะทำนายและผลเข้าไป
predicted_2 = model_2.predict_proba(test_match_features_2.values) # รับเป็น array อาทิตที่จะทำนายต่อไป

# Format the output into a DF with columns
predicted_table_2 = pd.DataFrame(predicted_2,columns=['draw', 'lose', 'win'])

# Compare predicted with test actual results 
predicted_table_2['predict_res'] = predicted_table_2.apply(predict_home_result, axis=1)
predicted_table_2['actual_res'] = model_test_matches['home_team_result']

# Straight comparison - count of equal / total to get %
(predicted_table_2[predicted_table_2['predict_res'] 
                 == model_test_matches['home_team_result']].count()) / model_test_matches['home_team_result'].count()

draw           0.464455
lose           0.464455
win            0.464455
predict_res    0.464455
actual_res     0.464455
dtype: float64

train model ข้อมูล team rating

In [33]:
predicted_2_1 = model_2.predict_proba(test_match_features_2_1.values) # รับเป็น array อาทิตที่จะทำนายต่อไป

# Format the output into a DF with columns
predicted_table_2_1 = pd.DataFrame(predicted_2_1,columns=['draw', 'lose', 'win'])

# Compare predicted with test actual results 
predicted_table_2_1['predict_res'] = predicted_table_2_1.apply(predict_home_result, axis=1)
predicted_table_2_1['actual_res'] = model_test_matches_1['home_team_result']

# Straight comparison - count of equal / total to get %
(predicted_table_2_1[predicted_table_2_1['predict_res'] 
                 == model_test_matches_1['home_team_result']].count()) / model_test_matches_1['home_team_result'].count()

draw           0.0
lose           0.0
win            0.0
predict_res    0.0
actual_res     0.0
dtype: float64

## Accuracy score of each model

In [34]:
# evaluate accuracy of prediction
# model.score(train_match_features.values, targets)
print('Accuracy of prediction (old match) = {:.2f}%'.format(model.score(train_match_features.values, targets) * 100))

print('Accuracy of prediction (team rating) = {:.2f}%'.format(model_2.score(train_match_features_2.values, targets) * 100))

Accuracy of prediction (old match) = 54.95%
Accuracy of prediction (team rating) = 44.88%


## Test data result from old match

In [35]:
compare_results = model_test_matches[['match_id', 'stage', 'home_team_goal', 
                                    'away_team_goal', 'home_team', 'away_team']].copy()
compare_results.rename(columns={'home_team_goal':'h_goal', 'away_team_goal':'a_goal'}, inplace=True)
compare_results = pd.concat([compare_results, predicted_table], axis=1)
compare_results = compare_results[['match_id', 'stage', 'h_goal', 'a_goal', 'home_team', 'predict_res', 'away_team','draw', 'lose', 'win', 'actual_res']]
compare_results = compare_results.rename(columns={'predict_res': 'predict_result'})
compare_results.tail(10)

,match_id,stage,h_goal,a_goal,home_team,predict_result,away_team,draw,lose,win,actual_res
412,7248,25,0.0,0.0,Stoke City,win,Watford,0.183688,0.257586,0.558726,draw
413,7247,25,1.0,1.0,Southampton,win,Brighton,0.333571,0.299372,0.367057,draw
414,7249,25,2.0,0.0,Tottenham Hotspur,win,Manchester United,0.248851,0.351756,0.399393,win
415,7246,25,1.0,1.0,Newcastle United,win,Burnley,0.186246,0.168556,0.645198,draw
416,7245,25,3.0,0.0,Manchester City,win,West Bromwich Albion,0.157521,0.093212,0.749268,win
417,7242,25,1.0,1.0,West Ham United,win,Crystal Palace,0.250269,0.277551,0.472180,draw
418,7243,25,0.0,3.0,Chelsea,win,Bournemouth,0.179292,0.094039,0.726669,lose
419,7241,25,3.0,1.0,Swansea City,lose,Arsenal,0.255235,0.549916,0.194849,win
420,7240,25,0.0,3.0,Huddersfield,lose,Liverpool,0.301479,0.470557,0.227964,lose
421,7244,25,2.0,1.0,Everton,win,Leicester City,0.270721,0.218525,0.510754,win


## Test data result from old team rating

In [36]:
compare_results_2 = model_test_matches[['match_id', 'stage', 'home_team_goal', 
                                    'away_team_goal', 'home_team', 'away_team']].copy()
compare_results_2.rename(columns={'home_team_goal':'h_goal', 'away_team_goal':'a_goal'}, inplace=True)
compare_results_2 = pd.concat([compare_results_2, predicted_table_2], axis=1)
compare_results_2 = compare_results_2[['match_id', 'stage', 'h_goal', 'a_goal', 'home_team', 'predict_res', 'away_team','draw', 'lose', 'win', 'actual_res']]
compare_results_2 = compare_results_2.rename(columns={'predict_res': 'predict_result'})
compare_results_2.tail(10)

,match_id,stage,h_goal,a_goal,home_team,predict_result,away_team,draw,lose,win,actual_res
412,7248,25,0.0,0.0,Stoke City,win,Watford,0.252377,0.298944,0.448679,draw
413,7247,25,1.0,1.0,Southampton,win,Brighton,0.250753,0.289748,0.459499,draw
414,7249,25,2.0,0.0,Tottenham Hotspur,win,Manchester United,0.253338,0.303400,0.443262,win
415,7246,25,1.0,1.0,Newcastle United,win,Burnley,0.253074,0.303650,0.443276,draw
416,7245,25,3.0,0.0,Manchester City,win,West Bromwich Albion,0.246363,0.267131,0.486505,win
417,7242,25,1.0,1.0,West Ham United,win,Crystal Palace,0.250785,0.289718,0.459497,draw
418,7243,25,0.0,3.0,Chelsea,win,Bournemouth,0.244306,0.258415,0.497278,lose
419,7241,25,3.0,1.0,Swansea City,win,Arsenal,0.256942,0.332093,0.410965,win
420,7240,25,0.0,3.0,Huddersfield,win,Liverpool,0.257398,0.336988,0.405615,lose
421,7244,25,2.0,1.0,Everton,win,Leicester City,0.250850,0.289657,0.459492,win


## Test data result from weight models

In [37]:
compare_results_3 = compare_results.copy()

# weight data
compare_results_3['draw'] = compare_results['draw'] * model_weight + compare_results_2['draw'] * (1 - model_weight)
compare_results_3['lose'] = compare_results['lose'] * model_weight + compare_results_2['lose'] * (1 - model_weight)
compare_results_3['win'] = compare_results['win'] * model_weight + compare_results_2['win'] * (1 - model_weight)
compare_results_3['predict_result'] = compare_results_3.apply(predict_home_result, axis=1)
compare_results_3.tail(10)

,match_id,stage,h_goal,a_goal,home_team,predict_result,away_team,draw,lose,win,actual_res
412,7248,25,0.0,0.0,Stoke City,win,Watford,0.197426,0.265858,0.536717,draw
413,7247,25,1.0,1.0,Southampton,win,Brighton,0.317008,0.297447,0.385545,draw
414,7249,25,2.0,0.0,Tottenham Hotspur,win,Manchester United,0.249748,0.342085,0.408167,win
415,7246,25,1.0,1.0,Newcastle United,win,Burnley,0.199612,0.195575,0.604813,draw
416,7245,25,3.0,0.0,Manchester City,win,West Bromwich Albion,0.175289,0.127995,0.696715,win
417,7242,25,1.0,1.0,West Ham United,win,Crystal Palace,0.250372,0.279984,0.469644,draw
418,7243,25,0.0,3.0,Chelsea,win,Bournemouth,0.192295,0.126914,0.680791,lose
419,7241,25,3.0,1.0,Swansea City,lose,Arsenal,0.255577,0.506351,0.238072,win
420,7240,25,0.0,3.0,Huddersfield,lose,Liverpool,0.292663,0.443843,0.263494,lose
421,7244,25,2.0,1.0,Everton,win,Leicester City,0.266747,0.232751,0.500502,win


In [38]:
# targets # ผลการแข่งขันจริงๆ
print('Total accuracy of prediction = {:.2f}%'.format(accuracy_score(compare_results_3['actual_res'], compare_results_3['predict_result']) * 100))

Total accuracy of prediction = 57.35%


## Predict next week from 2 model

#### From first model

In [39]:
compare_results_1_1 = model_test_matches_1[['match_id', 'stage', 'home_team_goal', 
                                    'away_team_goal', 'home_team', 'away_team']].copy()
compare_results_1_1.rename(columns={'home_team_goal':'h_goal', 'away_team_goal':'a_goal'}, inplace=True)
compare_results_1_1 = pd.concat([compare_results_1_1, predicted_table_1], axis=1)
compare_results_1_1 = compare_results_1_1[['match_id', 'stage', 'h_goal', 'a_goal', 'home_team', 'predict_res', 'away_team','draw', 'lose', 'win', 'actual_res']]
compare_results_1_1 = compare_results_1_1.rename(columns={'predict_res': 'predict_result'})
compare_results_1_1

,match_id,stage,h_goal,a_goal,home_team,predict_result,away_team,draw,lose,win,actual_res
0,7258,26,NaN,NaN,Watford,lose,Chelsea,0.228682,0.582857,0.188460,draw
1,7250,26,NaN,NaN,Bournemouth,win,Stoke City,0.294747,0.309211,0.396041,draw
2,7251,26,NaN,NaN,Arsenal,win,Everton,0.323540,0.122628,0.553832,draw
3,7252,26,NaN,NaN,Brighton,win,West Ham United,0.348475,0.251966,0.399559,draw
4,7253,26,NaN,NaN,Burnley,lose,Manchester City,0.225391,0.543231,0.231378,draw
5,7254,26,NaN,NaN,Crystal Palace,win,Newcastle United,0.127233,0.352827,0.519940,draw
6,7255,26,NaN,NaN,Leicester City,win,Swansea City,0.220163,0.201082,0.578755,draw
7,7256,26,NaN,NaN,Liverpool,win,Tottenham Hotspur,0.287926,0.296144,0.415930,draw
8,7257,26,NaN,NaN,Manchester United,win,Huddersfield,0.222226,0.227632,0.550142,draw
9,7259,26,NaN,NaN,West Bromwich Albion,win,Southampton,0.273481,0.355934,0.370585,draw


#### From second model

In [40]:
compare_results_2_1 = model_test_matches_1[['match_id', 'stage', 'home_team_goal', 
                                    'away_team_goal', 'home_team', 'away_team']].copy()
compare_results_2_1.rename(columns={'home_team_goal':'h_goal', 'away_team_goal':'a_goal'}, inplace=True)
compare_results_2_1 = pd.concat([compare_results_2_1, predicted_table_2_1], axis=1)
compare_results_2_1 = compare_results_2_1[['match_id', 'stage', 'h_goal', 'a_goal', 'home_team', 'predict_res', 'away_team','draw', 'lose', 'win', 'actual_res']]
compare_results_2_1 = compare_results_2_1.rename(columns={'predict_res': 'predict_result'})
compare_results_2_1

,match_id,stage,h_goal,a_goal,home_team,predict_result,away_team,draw,lose,win,actual_res
0,7258,26,NaN,NaN,Watford,win,Chelsea,0.257011,0.332026,0.410963,draw
1,7250,26,NaN,NaN,Bournemouth,win,Stoke City,0.253804,0.308324,0.437873,draw
2,7251,26,NaN,NaN,Arsenal,win,Everton,0.250915,0.289597,0.459488,draw
3,7252,26,NaN,NaN,Brighton,win,West Ham United,0.254500,0.313025,0.432475,draw
4,7253,26,NaN,NaN,Burnley,win,Manchester City,0.257501,0.336886,0.405613,draw
5,7254,26,NaN,NaN,Crystal Palace,win,Newcastle United,0.251548,0.294361,0.454091,draw
6,7255,26,NaN,NaN,Leicester City,win,Swansea City,0.249925,0.285167,0.464908,draw
7,7256,26,NaN,NaN,Liverpool,win,Tottenham Hotspur,0.254003,0.308133,0.437864,draw
8,7257,26,NaN,NaN,Manchester United,win,Huddersfield,0.242136,0.249849,0.508015,draw
9,7259,26,NaN,NaN,West Bromwich Albion,win,Southampton,0.252377,0.298944,0.448679,draw


## Last result

In [41]:
compare_results_3_1 = compare_results_1_1.copy()

# weight data
compare_results_3_1['draw'] = compare_results_1_1['draw'] * model_weight + compare_results_2_1['draw'] * (1 - model_weight)
compare_results_3_1['lose'] = compare_results_1_1['lose'] * model_weight + compare_results_2_1['lose'] * (1 - model_weight)
compare_results_3_1['win'] = compare_results_1_1['win'] * model_weight + compare_results_2_1['win'] * (1 - model_weight)
compare_results_3_1['predict_result'] = compare_results_3_1.apply(predict_home_result, axis=1)
compare_results_3_1.drop('actual_res', axis=1, inplace=True)
compare_results_3_1

,match_id,stage,h_goal,a_goal,home_team,predict_result,away_team,draw,lose,win
0,7258,26,NaN,NaN,Watford,lose,Chelsea,0.234348,0.532691,0.232961
1,7250,26,NaN,NaN,Bournemouth,win,Stoke City,0.286558,0.309034,0.404408
2,7251,26,NaN,NaN,Arsenal,win,Everton,0.309015,0.156022,0.534963
3,7252,26,NaN,NaN,Brighton,win,West Ham United,0.329680,0.264178,0.406142
4,7253,26,NaN,NaN,Burnley,lose,Manchester City,0.231813,0.501962,0.266225
5,7254,26,NaN,NaN,Crystal Palace,win,Newcastle United,0.152096,0.341134,0.506770
6,7255,26,NaN,NaN,Leicester City,win,Swansea City,0.226116,0.217899,0.555985
7,7256,26,NaN,NaN,Liverpool,win,Tottenham Hotspur,0.281142,0.298542,0.420317
8,7257,26,NaN,NaN,Manchester United,win,Huddersfield,0.226208,0.232075,0.541717
9,7259,26,NaN,NaN,West Bromwich Albion,win,Southampton,0.269260,0.344536,0.386203
